### 열을 행 방향으로 이동

Shift

In [54]:
import pandas as pd


In [55]:

data = [
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가': 100, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  99, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 
    {'시가': 100, '고가': 110, '저가': 90, '종가': 105}, 
    {'시가': 100, '고가': 112, '저가': 80, '종가':  95}, 
    {'시가':  99, '고가': 115, '저가': 70, '종가':  85}, 
    {'시가':  70, '고가':  80, '저가': 60, '종가':  75}, 

]

indexes = ['20200615', '20200616', '20200717', '20200718'
           , '20200815', '20200816', '20200917', '20200918']

dt_first = pd.DatetimeIndex(indexes)
df_first = pd.DataFrame(data, index=dt_first)
df_first.info(), df_first.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8 entries, 2020-06-15 to 2020-09-18
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   시가      8 non-null      int64
 1   고가      8 non-null      int64
 2   저가      8 non-null      int64
 3   종가      8 non-null      int64
dtypes: int64(4)
memory usage: 320.0 bytes


(None,
              시가   고가  저가   종가
 2020-06-15  100  110  90  105
 2020-06-16  100  112  80   95
 2020-07-17   99  115  70   85
 2020-07-18   70   80  60   75
 2020-08-15  100  110  90  105)

#### 전날 종가와 차이

In [56]:
df_first['전일종가'] = df_first['종가'].shift(periods=1)
df_first['삼일전종가'] = df_first['종가'].shift(periods=3) # 하고 나면 float? 
df_first.head()

,시가,고가,저가,종가,전일종가,삼일전종가
2020-06-15,100,110,90,105,NaN,NaN
2020-06-16,100,112,80,95,105.0,NaN
2020-07-17,99,115,70,85,95.0,NaN
2020-07-18,70,80,60,75,85.0,105.0
2020-08-15,100,110,90,105,75.0,95.0


In [57]:
condition = df_first['전일종가'] < df_first['종가']
condition

2020-06-15    False
2020-06-16    False
2020-07-17    False
2020-07-18    False
2020-08-15     True
2020-08-16    False
2020-09-17    False
2020-09-18    False
dtype: bool

In [58]:
df_first[condition]

,시가,고가,저가,종가,전일종가,삼일전종가
2020-08-15,100,110,90,105,75.0,95.0


In [59]:
df_first['등락'] = df_first['종가'] - df_first['전일종가']

In [60]:
df_first[df_first['등락']>0]

,시가,고가,저가,종가,전일종가,삼일전종가,등락
2020-08-15,100,110,90,105,75.0,95.0,30.0


### 수익률 계산 with shift

In [61]:
df_first['종가'] / df_first['시가']

2020-06-15    1.050000
2020-06-16    0.950000
2020-07-17    0.858586
2020-07-18    1.071429
2020-08-15    1.050000
2020-08-16    0.950000
2020-09-17    0.858586
2020-09-18    1.071429
dtype: float64

### rolling()
- shift와 연산 가능

In [62]:
# 샘플 데이터 생성 
data = { 'value': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] }

In [63]:
df_second = pd.DataFrame(data)
df_second

,value
0,10
1,20
2,30
3,40
4,50
5,60
6,70
7,80
8,90
9,100


In [64]:
df_second['rolling3'] = df_second['value'].rolling(window=3).mean()
df_second['rolling6'] = df_second['value'].rolling(window=6).mean()
df_second

,value,rolling3,rolling6
0,10,NaN,NaN
1,20,NaN,NaN
2,30,20.0,NaN
3,40,30.0,NaN
4,50,40.0,NaN
5,60,50.0,35.0
6,70,60.0,45.0
7,80,70.0,55.0
8,90,80.0,65.0
9,100,90.0,75.0


### resampling()
- down sampling (up sampling 도 있다)
- 컬럼별 정책 줄 수 있다
- shift+groupby 느낌

In [65]:
df_resample_first = pd.read_excel('../datasets/ss_ex_1.xlsx', index_col=0)
df_resample_first.head()

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2021/08/13,74400,-2600,-3.38,75800,76000,74100,61270643,4575267536355,444151821720000,5969782550
2021/08/12,77000,-1500,-1.91,77100,78200,76900,42365223,3276635421700,459673256350000,5969782550
2021/08/11,78500,-1700,-2.12,79600,79800,78500,30241137,2389977254924,468627930175000,5969782550
2021/08/10,80200,-1300,-1.60,82300,82400,80100,20362639,1643107615500,478776560510000,5969782550
2021/08/09,81500,0,0.00,81500,82300,80900,15522581,1267668377900,486537277825000,5969782550


In [66]:
df_resample_first.info()

<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 2021/08/13 to 2021/02/15
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      127 non-null    int64  
 1   대비      127 non-null    int64  
 2   등락률     127 non-null    float64
 3   시가      127 non-null    int64  
 4   고가      127 non-null    int64  
 5   저가      127 non-null    int64  
 6   거래량     127 non-null    int64  
 7   거래대금    127 non-null    int64  
 8   시가총액    127 non-null    int64  
 9   상장주식수   127 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 10.9+ KB


In [67]:
df_resample_first.index

Index(['2021/08/13', '2021/08/12', '2021/08/11', '2021/08/10', '2021/08/09',
       '2021/08/06', '2021/08/05', '2021/08/04', '2021/08/03', '2021/08/02',
       ...
       '2021/02/26', '2021/02/25', '2021/02/24', '2021/02/23', '2021/02/22',
       '2021/02/19', '2021/02/18', '2021/02/17', '2021/02/16', '2021/02/15'],
      dtype='object', name='일자', length=127)

In [68]:
df_resample_first.index = pd.to_datetime(df_resample_first.index)
df_resample_first.index

DatetimeIndex(['2021-08-13', '2021-08-12', '2021-08-11', '2021-08-10',
               '2021-08-09', '2021-08-06', '2021-08-05', '2021-08-04',
               '2021-08-03', '2021-08-02',
               ...
               '2021-02-26', '2021-02-25', '2021-02-24', '2021-02-23',
               '2021-02-22', '2021-02-19', '2021-02-18', '2021-02-17',
               '2021-02-16', '2021-02-15'],
              dtype='datetime64[ns]', name='일자', length=127, freq=None)

#### 월 단위 resampling

In [75]:
df_resample_second = df_resample_first.resample('MS').first() # 묶는 index중 처음
df_resample_second = df_resample_second.loc[:,:"거래량"]
df_resample_second

,종가,대비,등락률,시가,고가,저가,거래량
일자,,,,,,,
2021-02-01,84200,2600,3.19,83800,84500,83300,23529706
2021-03-01,83600,1100,1.33,85100,85300,83000,33498180
2021-04-01,82900,1500,1.84,82500,83000,82000,18676461
2021-05-01,81700,200,0.25,81000,82400,81000,15710336
2021-06-01,80600,100,0.12,80500,81300,80100,14058401
2021-07-01,80100,-600,-0.74,80500,80600,80000,13382882
2021-08-01,79300,800,1.02,79200,79500,78700,11739124


In [85]:
how = {
    '시가':'first',
    '종가':'last',
    '고가': 'max',
    '저가': 'min',
    '거래량': 'sum',
    '등락률': 'mean'
}
# import numpy as np
# how = {
#     '시가':'first',
#     '종가':'last',
#     '고가': np.max,
#     '저가': np.min,
#     '거래량': np.sum,
#     '등락률': np.mean
# }
df_resample_first.resample('MS').apply(how)

,시가,종가,고가,저가,거래량,등락률
일자,,,,,,
2021-02-01,83800,82500,86000,81000,2.550207e+07,0.133000
2021-03-01,85100,81400,85300,80600,1.761874e+07,-0.055909
2021-04-01,82500,81500,86200,81500,1.695174e+07,0.009545
2021-05-01,81000,80500,83500,78400,1.853742e+07,-0.059474
2021-06-01,80500,80700,83000,79600,1.514088e+07,0.015455
2021-07-01,80500,78500,81300,78100,1.254028e+07,-0.121364
2021-08-01,79200,74400,83300,74100,2.633112e+07,-0.520000


#### 일 단위 resampling

In [89]:
how = {
    '시가':'first',
    '종가':'last',
    '고가': 'max',
    '저가': 'min',
    '거래량': 'sum',
    '등락률': 'mean'
}

df_resample_third = df_resample_first.resample('3D').apply(how)
df_resample_third.head()

,시가,종가,고가,저가,거래량,등락률
일자,,,,,,
2021-02-15,83800.0,83200.0,86000.0,83000.0,62320541,0.673333
2021-02-18,83200.0,82600.0,83600.0,81000.0,47208562,-0.355000
2021-02-21,83800.0,82000.0,84200.0,81100.0,46007200,-0.360000
2021-02-24,81800.0,82500.0,85400.0,81300.0,99484437,0.246667
2021-02-27,NaN,NaN,NaN,NaN,0,NaN


In [91]:
df_resample_first.resample('12H').apply(how)
# 현상태에서 upsampling 이 안된다 중간 값을 채워야함. 

/tmp/ipykernel_44569/906870948.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resample_first.resample('12H').apply(how)


,시가,종가,고가,저가,거래량,등락률
일자,,,,,,
2021-02-15 00:00:00,83800.0,84200.0,84500.0,83300.0,23529706,3.19
2021-02-15 12:00:00,NaN,NaN,NaN,NaN,0,NaN
2021-02-16 00:00:00,84500.0,84900.0,86000.0,84200.0,20483100,0.83
2021-02-16 12:00:00,NaN,NaN,NaN,NaN,0,NaN
2021-02-17 00:00:00,83900.0,83200.0,84200.0,83000.0,18307735,-2.00
...,...,...,...,...,...,...
2021-08-11 00:00:00,79600.0,78500.0,79800.0,78500.0,30241137,-2.12
2021-08-11 12:00:00,NaN,NaN,NaN,NaN,0,NaN
2021-08-12 00:00:00,77100.0,77000.0,78200.0,76900.0,42365223,-1.91


### 수익률

In [106]:
data = {'삼성전자': [52200, 52300, 52900, 52000, 51700], 
           'LG전자': [68200, 67800, 68800, 67500, 66300]}

In [107]:
df_first = pd.DataFrame(data)

In [108]:
pd.DataFrame(data).pct_change(periods=2) # 이외에 이평선,지평선 등 여러 지수 함수들이 이미 있다

,삼성전자,LG전자
0,NaN,NaN
1,NaN,NaN
2,0.013410,0.008798
3,-0.005736,-0.004425
4,-0.022684,-0.036337


#### 수익률 누적 곱

In [109]:
df_first.cumprod()

,삼성전자,LG전자
0,52200,68200
1,2730060000,4623960000
2,144420174000000,318128448000000
3,7509849048000000000,3026926166290448384
4,-7873481838642413568,3076047170515828736


In [110]:
(1 + pd.DataFrame(data).pct_change(periods=1)).cumprod()

,삼성전자,LG전자
0,NaN,NaN
1,1.001916,0.994135
2,1.013410,1.008798
3,0.996169,0.989736
4,0.990421,0.972141


In [102]:
data = {"DailyReturn": [0.01, 0.02, -0.005, 0.015]}


In [103]:
df_second = pd.DataFrame(data)
df_second

,DailyReturn
0,0.010
1,0.020
2,-0.005
3,0.015


In [104]:
(1 + df_second['DailyReturn']).cumprod() # 분수 누적 곱은 값이 작아지기 때문에 그리고 수익률 구할 때는 합이 맞음

0    1.010000
1    1.030200
2    1.025049
3    1.040425
Name: DailyReturn, dtype: float64